In [2]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00


In [ ]:
# 매장과 제품군에 맞는 파라미터 조정 추가
import pandas as pd
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

# 파일 경로 업데이트
train_path = '/content/drive/MyDrive/Dev_course/3차 팀 프로젝트/data_pre.csv'
test_path = '/content/drive/MyDrive/Dev_course/3차 팀 프로젝트/test.csv'

# 데이터 로드
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# 매장 번호의 범위를 지정
store_numbers = range(1, 55)

# 예측 결과를 담을 리스트
all_predictions = []

# 모든 매장 번호에 대해 반복
for store_nbr in store_numbers:
    # 매장 번호에 포함된 모든 제품군을 추출
    unique_families = train[train['store_nbr'] == store_nbr]['family'].unique()

    # 모든 제품군에 대해 반복
    for family in unique_families:
        # 해당 제품군의 데이터를 필터링
        train_filtered = train[(train['store_nbr'] == store_nbr) & (train['family'] == family)]
        train_filtered['date'] = pd.to_datetime(train_filtered['date'])
        train_filtered = train_filtered.set_index('date').sort_index()

        # auto_arima를 사용해 각 매장과 제품군별로 최적의 파라미터를 찾음
        auto_arima_model = auto_arima(
            train_filtered['sales'],
            start_p=1, start_q=1,
            max_p=3, max_q=3,
            seasonal=False,
            d=None,
            trace= False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=False
        )
        best_p, best_d, best_q = auto_arima_model.order

        print(f"Store {store_nbr}, Family '{family}' - Parameters: p={best_p}, d={best_d}, q={best_q}")

        # 매장과 제품군별로 적절한 p, d, q 값을 추출한 후에 모델 생성
        arima_model = ARIMA(train_filtered['sales'], order=(best_p, best_d, best_q))
        arima_fitted = arima_model.fit()

        # 테스트 데이터를 필터링
        test_filtered = test[(test['store_nbr'] == store_nbr) & (test['family'] == family)]
        test_filtered['date'] = pd.to_datetime(test_filtered['date'])
        test_filtered = test_filtered.set_index('date').sort_index()

        # 예측 결과 생성
        forecast_sales = arima_fitted.forecast(steps=len(test_filtered))

        # 음수를 방지하기 위해 예측값을 0보다 작지 않게 조정
        forecast_sales = forecast_sales.clip(lower=0)

        # 테스트 데이터에 예측 결과 추가
        test_filtered['predicted_sales'] = forecast_sales.values

        # 원래 데이터 프레임에서 'id'를 유지하기 위해 인덱스 재설정
        test_filtered.reset_index(inplace=True)

        # 예측된 id와 sales 값을 추출하여 결합
        all_predictions.append(test_filtered[['id', 'predicted_sales']].rename(columns={'predicted_sales': 'sales'}))

# 모든 예측 결과를 하나의 데이터프레임으로 결합
submission = pd.concat(all_predictions)

# 결과를 CSV 파일로 저장
output_path = '/content/drive/MyDrive/Dev_course/3차 팀 프로젝트/submission_0510.csv'
submission.to_csv(output_path, index=False)
